In [1]:
# 
# !pip install hdfs
# ref : https://hdfscli.readthedocs.io/en/latest/quickstart.html#python-bindings

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from hdfs import InsecureClient #hdfs 연결 담당 모듈 - 클라이언트 프로그램 구성시
import requests #api 요청시 사용
import json #api 다운받은 data와 vaccine data json 처리시 사용
import datetime as dt

## 클라이언트 객체 생성
- hdfs 공간 사용을 위해서는 클라이언트 객체를 생성해야한다.
- hdfs 연결 담당
- read/write/append/update 등과 관련된 모듈도 포함하고 있다.
- InsecureClient 모듈 사용해서 만들기
    - InsecureClient(연결주소, user=사용자)
- 파일읽기
    - InsecureClient.read(파일명(파일경로포함))
        - 해당 파일을 읽을 수 있는 reader 객체를 반환
- 파일쓰기
    - InsecureClient.write(저장될파일명(파일경로포함))
        - 해당 파일에 쓸 수 있는 reader 객체를 반환
    - InsecureClient.write(저장될파일명(파일경로 포함),'추가될 내용',append=True)
        - 기존파일에 추가될내용을 append(파일 끝에 쓰기)        
    - InsecureClient.upload(hdfs에 복사될 폴더명(경로포함), 원본 폴더명(경로포함))
        - 지정된 원본 폴더 파일을 저장할 hsdf 폴더에 복사
        - 폴더를 생성하고 시작 : 기존에 동일 이름의 폴더가 있으면 에러발생함

In [3]:
# port 9870: name node 연결 port
# port 9000: data node(hdfs) 연결 port
client = InsecureClient('http://localhost:9870', user='root')

## hdfs로부터 읽기
- 파일 읽어올 때 필요한 순서
1) 파일과 연결(open)
2) 파일읽기
3) 파일 연결 끊기(close): 개발자가 close해야 자원 정리가 가능
    - with문 사용하면 자동으로 open~close를 진행한다.

In [5]:
# 파일 읽기위한 reader 객체 생성 후 진행
with client.read('/rdd/score.txt') as reader : 
    score = reader.read()

In [11]:
# byte code로 읽어옴 (low-level로 읽어오기 때문)
score
# byte code data를 decode해서 저장 후 확인할 수 있다.
# https://docs.python.org/ko/3/library/stdtypes.html#bytes-and-bytearray-operations
# bytes.decode(data)
score_data = bytes.decode(score)
print(score_data)

b'\xea\xb9\x80\xec\xb2\xa0\xec\x88\x98 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 50\r\n\xed\x99\x8d\xea\xb8\xb8\xeb\x8f\x99 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 80\r\n\xec\x9e\x84\xea\xba\xbd\xec\xa0\x95 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 60\r\n\xec\x9e\x84\xec\x9a\x94\xed\x99\x98 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 100\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xec\x9d\xb4\xec\x9c\xa4\xec\x97\xb4 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 90\r\n'

김철수 스파크 50
홍길동 스파크 80
임꺽정 스파크 60
임요환 텐서플로우 100
홍진호 텐서플로우 22
홍진호 텐서플로우 22
이윤열 텐서플로우 90



## hdfs에 쓰기

In [21]:
# hdd(note 디렉터리)에 있는 data(file)를 hdfs로 복사
# 1. 로컬데이터 파일 열기: open() python
# 2. hdfs client 통해서 쓰기 객체 with등록
# 3. open 객체 이용해서 한 줄 씩 읽어서 -> hdfs에 쓰기
# 현재위치: /root/note
# 복사할 sido_area.csv 윈도우 10버전에서 생성, encoding='CP949'
with open('data/corona_data/sido_area.csv', encoding='CP949') as reader, client.write('/corona_data/sido_area_tmp.csv') as writer:
    for line in reader: # 읽어와서
        writer.write(line.encode('CP949')) #원래 encoding 방식 유지 시켜야 함, 바로쓰기

### 인구사회 data를 D.W에 저장
- 현재 Linux FS에 있는 파일을 HDFS로 가져오기
- 인구, 면적, 다중이용시설 data

In [13]:
# 폴더 전송: 폴더 내에 있는 파일을 hdfs 특정 디렉터리에 모두 복사
# 해당 모듈은 디렉터리와 파일부터 생성하고 쓰기 진행
# 프로그램으로 파일 한줄씩 읽어와서 바로 hdfs파일에 쓰기
# client.upload(dest, source)
client.upload('/corona_data/tmp', 'data/corona_data')

'/corona_data/tmp'

## hdfs에 수정하기
- data 추가이므로 data형식과 encoding방식은 기존 파일과 동일하게 구성

In [15]:
client.write('/rdd/score.txt', '최연성 장고 100'.encode('UTF-8'), append=True)

In [16]:
# 수정된 파일 읽어와서 확인
with client.read('/rdd/score.txt') as reader :
    score = reader.read()
score_data = bytes.decode(score)
print(score_data)

김철수 스파크 50
홍길동 스파크 80
임꺽정 스파크 60
임요환 텐서플로우 100
홍진호 텐서플로우 22
홍진호 텐서플로우 22
이윤열 텐서플로우 90
최연성 장고 100


## hdfs 권한 수정
- 리눅스 명령어(chmod)
- InsecureClient클래스의 set_permission('권한변경할파일(폴더도 파일임)','권한문자')
    - 권한문자: 10진수 권한
    - eg) 777 : rwxrwxrwx : 모든 user가 읽기/쓰기/실행 권한을 갖는다
          777권한은 temp폴더 등이 갖는 권한(완전 풀권한이라 위험,, 아무나 들어갈 수 있으니까 공격 대상이 될 수 있음)

In [18]:
client.set_permission('/corona_data/tmp', '777')

## hdfs 삭제
- InsecureClient클래스의 delete('삭제할파일(경로 포함)')
    - 삭제가 정상적으로 진행되면 True반환
    - False반환인 경우 대부분 파일명 문제

In [22]:
client.delete('/corona_data/tmp/sido_area.xlsx')

True

################################################################### 

## 실습 Data Pipeline
- Data Warehouse Extract
    - 현재 실행중인 ubuntu server의 HDD -> Hadoop File System으로 변경된 공간
        - hadoop 설치 & hdfs 사용가능하도록 구성 완료
    - Extract 과정을 거쳐 hdfs에 data가 저장됨
        1. 코로나 발생현황(일간데이터): API로 수집 (json형식)
        2. 코로나 백신 접종 현황(일간데이터): 파일로 보관중, 파일데이터에서 특정 일에 대한 데이터를 json형식으로 변환하여 수집 예정
        3. 인구 사회 데이터: 파일데이터(변환없이 사용)
- Transformation
    - HDFS에서 데이터 load해와서(RAM에다가 ㅇㅇ) spark 모듈 이용해서 transformation진행
    - W.H의 데이터보다 정형화된 데이터로 transformation

- Data Mart Load
    - transformation 진행된 데이터를 MySQL DB에 저장(적재)
- 한번더 Transformation
    - DM에서 데이터 load해와서 spark 모듈 이용해서 transformation진행
    - DM의 데이터보다 정형화되고 논리적인 의미를 포함하는 데이터로 transformation
- 운영 DB Load
    - transformation 진행된 데이터를 MySQL DB에 저장(적재)
    - 운영DB에 적재되는 데이터는 서비스와 연결됨
    - 정형화되고 논리적 의미를 포함해야함
        - 단순 결합이 아니라, 데이터의 조합을 통해 유의미한 결과를 보일 수 있어야함

## REST_API로 데이터를 호출해 HDFS에 저장

#### API 데이터 수집 과정
1. 데이터 이용 신청
2. site에서 데이터 수집을 위한 url을 알아와야함
3. site에서 가이드하고 있는 방식(get/post)으로 데이터 요청 진행
    - header와 parameters를 요구하게 됨
        - 생성해서 요청시마다 사용
        - params에 service key를 포함시켜서 구성
4. 요청된 데이터가 프로그램 변수에 저장되면 이 data를 정리해서 저장
5. excuteRestApi() 요청하고 데이터 받아서 변수저장 함수를 호출한 곳에 데이터 반환(그래서 return 문이 있다.)
   - json형태의 text 객체를 반환

In [3]:
# 데이터 요청에 필요한 함수 인수를 받아서
# 데이터 요청하고 전송된 데이터를 반환하는 함수
# 데이터 요청이 실패했을 경우 예외 발생시킴
# log를 구성해서 예외 발생 시 log에 기록되도록 진행

def executeRestApi(method, url, headers, params):  
    # params, headers는 dict로 구성되어야 함
    # raise Exception('응답코드 : 500')  # 예외 테스트
    # err_num = 10/0 # 예외 테스트
    if method == "get": # header가 필요없음 - 파라미터로 사용해야 된다면 보통 {}(빈딕셔너리)가 전달됨
        res = requests.get(url, params=params, headers=headers)
    else: # post 요청: header필요, header는 {}로 구성되어야함
        res = requests.post(url, data=params, headers=headers)

    if res == None or res.status_code != 200:
        raise Exception('응답코드 : ' + str(res.status_code))
       
    return res.text



### 기준일자 함수
- 요청 data의 기준이 되는 yyyy-mm-dd 형식의 날짜형태 문자열을 생성하는 함수
- 현 시점으로부터 며칠전 data인지 기준으로 날짜 생성
    - eg) 어제data: 오늘로부터 1일전
    - 이 기간을 인수로 전달하면 됨
- 위에서 import datetime as dt로 패키지 import했었다.

In [4]:
dt.datetime.now() - dt.timedelta(365)
d = dt.datetime.now()
d.year
d.month
type(d.year)

datetime.datetime(2024, 1, 21, 1, 23, 10, 757505)

2025

1

int

In [5]:
# 오늘부터 얼마나 이전 날짜인지
def cal_std_day(before_day):
    x = dt.datetime.now() - dt.timedelta(before_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month) #원래 int타입이라 str로 변환
    day = x.day if x.day >= 10 else '0' + str(x.day)
    return str(year)+'-'+str(month)+'-'+str(day) #if에 안걸리면 int타입으로 반환되니까 str필요

In [6]:
cal_std_day(1)
cal_std_day(365*3) #오늘부터 3년전
cal_std_day(0)
cal_std_day(-1)
cal_std_day(365*3+3)

'2025-01-19'

'2022-01-21'

'2025-01-20'

'2025-01-21'

'2022-01-18'

## log 기록

#### logger
- log 기록 프로그램
- log : 프로그램을 실행하면서 나타나는 일련의 사건(error, event등 물리적으로 발생하는 내용이나 네트워크 송수신중 발생하는 이벤트)을 기록하는 프로그램
- 파이썬 패키지 : logging

- log는 외부연결 시 오류 관련 정보를 확인하기 위해 사용자 정의를 기록해야한다.
- restAPI는 외부서버: log 기록
- 사용자 정의 log 생성 순서
    1. 로그 저장할 dir필요
    2. 로그 관련 패키지 import
    3. 모듈 활용 객체 logger생성 (logging.getLogger('로거이름'))
    4. 로그파일 핸들러 생성 (logging.FileHandler('로그파일명.log'))
    5. 로그파일 핸들러를 로거에 추가(로거와 로그파일 연결) : logger.addHandler(로그파일 핸들러 객체명)

In [7]:
import logging #로그 관련 패키지 import

In [8]:
co_logger = logging.getLogger('corona_api')
f_handler = logging.FileHandler('./log/rest_api/'+cal_std_day(0)+'.log')
co_logger.addHandler(f_handler) # log를 기록할 파일에 연결된 logger가 생성
# note아래 log/rest_api라는 폴더가 없어서 에러가 났었다. vscode에서 dir생성해줌
# dir만들고 들어가보면 오늘날짜.log라는 파일이 생성되어있다. -> 에러 없어서 현재는 빈파일로 확인됨

In [9]:
# 강제 error 발생시켜서 log에 기록하는 test진행
    # 실재 프로그램에서는 예외 또는 에러가 발생했을 때 기록을 위한 logger error발생
# 이렇게 문자열로 찍을 수도 있음. json양식으로 log를 저장하면 다음에 읽어와서 처리하기도 편하다.
co_logger.error('corona_patient_'+cal_std_day(0)+'.json 다운로드실패')
# 실행하고 가보면 "corona_patient_2025-01-17.json 다운로드실패" 라고 한 줄 찍혀있음
# 발생된 에러의 log는 /root/note/log/rest_api/ 디렉터리 내 log 파일에서 확인해야 함

## api 호출

In [10]:
# base var 생성
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api'
service_key = 'UlT9VwHvrFyq4AtJdmGh0byLAh3I/aaMbqLw9Lq1uigvSei2yTpPyBnrNKIbJlLV7oBc1hcnfI+ri7nq9XdcOg=='
# 요청해 응답받은 rest_api data를 hdfs에 저장할 dir
file_dir = '/corona_data/patient/'

- 기본함수
    - executeRestApi(method, url, headers, params)
        - 데부분의 요청에 적용되는 함수
    - params는 여러 data를 설정해야함
        - 파라미터 생성 함수: create_param(std_day)
            - std_day: 날짜가 아닌 오늘부터 얼마나 이전 데이터를 요청할 것인가에 대한 수치
            - 이번 코로나발생현황 공공data api에만 적용되는 함수

In [12]:
# 데이터별로 요청변수가 상이함(api 데이터가 다르면 당연히 return내용이 달라져야함)
def create_param(std_day) :
    return {'serviceKey':service_key
        ,'pageNo':1
        ,'numOfRows':'500'
        ,'apiType':'JSON'
        ,'std_day':cal_std_day(std_day)}

In [14]:
create_param(365*3+3)

{'serviceKey': 'UlT9VwHvrFyq4AtJdmGh0byLAh3I/aaMbqLw9Lq1uigvSei2yTpPyBnrNKIbJlLV7oBc1hcnfI+ri7nq9XdcOg==',
 'pageNo': 1,
 'numOfRows': '500',
 'apiType': 'JSON',
 'std_day': '2022-01-18'}

In [15]:
# 필요변수 생성
params = create_param(365*3)
# log기록 내용은 개발자가 결정, 형식은dict
    #json형식(dict)로 log기록하면 분석이 쉬워짐(단순 문자열보다 형식이 있는 데이터 구성해야 나중에 분석하기가 좋음)
log_dict={
    "is_success":"Fail"
    ,"type":"corona_patient"
    ,"std_day":params['std_day']
    ,"params":params
}

In [30]:
# 요청해서 json응답받고 hdfs에 저장 - 날짜별로 파일명 구분
# 외부파일시스템과 연결하는 경우 try-except구문을 반드시 넣어야함
try : 
    res=executeRestApi('get',url,{},params) #api에 data요청 후 결과 응답(res변수에 json형식으로 반환)
    file_name='corona_patient_'+cal_std_day(365*3)+'.json' #hdfs에 저장할 파일명 생성
    client.write(file_dir+file_name,res,encoding='utf-8') #hdfs에 저장
except Exception as e: # 일반 python 예외
    log_dict['err_msg']= e.__str__()
    log_json = json.dumps(log_dict, ensure_ascii=False) #log json 형태로 기록
    co_logger.error(log_json) #logger가 발생시키는 에러 - log 기록을 위함  
print(res)

{"pageNo":"1","resultCode":"00","totalCount":19,"items":[{"gubunEn":"Incheon","deathCnt":"326","defCnt":"40625","isolClearCnt":"35422","stdDay":"2022-01-18","localOccCnt":"177","qurRate":"1378","overFlowCnt":"19","gubunCn":"仁川","incDec":"196","isolIngCnt":"0","gubun":"인천"},{"gubunEn":"Chungcheongbuk-do","deathCnt":"123","defCnt":"12555","isolClearCnt":"11766","stdDay":"2022-01-18","localOccCnt":"55","qurRate":"786","overFlowCnt":"4","gubunCn":"忠北","incDec":"59","isolIngCnt":"0","gubun":"충북"},{"gubunEn":"Gangwon-do","deathCnt":"110","defCnt":"13838","isolClearCnt":"12657","stdDay":"2022-01-18","localOccCnt":"94","qurRate":"899","overFlowCnt":"0","gubunCn":"江原","incDec":"94","isolIngCnt":"0","gubun":"강원"},{"gubunEn":"Busan","deathCnt":"344","defCnt":"28102","isolClearCnt":"25796","stdDay":"2022-01-18","localOccCnt":"102","qurRate":"839","overFlowCnt":"12","gubunCn":"釜山","incDec":"114","isolIngCnt":"0","gubun":"부산"},{"gubunEn":"Ulsan","deathCnt":"65","defCnt":"7459","isolClearCnt":"7107",

## 6일간의 발생현황 API로 받아서 데이터 hdfs에 저장

- 날짜별 발생현황 파일 요청
    - 해당 파일에 날짜를 파일명에 포함시켜서
    - 정리없이 hdfs저장

In [44]:
# ETL 연습용 data hdfs에 저장
for i in range(365*3, 367*3):
    params = create_param(i)
    print(params['std_day'])
    try : 
        res=executeRestApi('get',url,{},params)
        file_name='corona_patient_'+cal_std_day(i)+'.json' #hdfs에 저장할 파일명 생성
        client.write(file_dir+file_name,res,encoding='utf-8') #hdfs에 저장
    except Exception as e: # 일반 python 예외
        log_dict['err_msg']= e.__str__()
        log_json = json.dumps(log_dict, ensure_ascii=False) #log json 형태로 기록
        co_logger.error(log_json) #logger가 발생시키는 에러 - log 기록을 위함  

2022-01-18
2022-01-17
2022-01-16
2022-01-15
2022-01-14
2022-01-13


- 2022-01-13 ~ 2022-01-18 코로나 발생현황 data hdfs에 저장
- json 형식으로 저장

## corona_vaccine 접종현황 data
- 파일로 제공
- 파일 data 읽어서 json형식으로 변경한 후에 hdfs에 저장
- 필요 data 추출 후 정리 & 형식 변환

In [17]:
import pandas as pd

In [18]:
# 인구data
pop = pd.read_csv('data/corona_data/sido_population.csv',encoding='CP949')
pop.head(2)

,loc,total,male,female
0,전국,52739009,26360035,26378974
1,서울,9736027,4721977,5014050


In [19]:
vac_v1 = pd.read_csv('data/corona_data/Covid19Vaccine.csv', index_col=0)
vac_v1.head(5)

,sido,firstCnt,secondCnt,totalFirstCnt,totalSecondCnt,accumulatedFirstCnt,accumulatedSecondCnt,id,thirdCnt,totalThirdCnt,accumulatedThirdCnt
baseDate,,,,,,,,,,,
2022-11-09 00:00:00,제주특별자치도,6.0,5.0,591064,585442.0,591058,585437.0,10176,38.0,439144.0,439106.0
2022-11-09 00:00:00,전국,175.0,224.0,45123225,44693321.0,45123050,44693097.0,10159,1678.0,33673146.0,33671468.0
2022-11-09 00:00:00,서울특별시,28.0,43.0,8335326,8257811.0,8335298,8257768.0,10160,482.0,6076911.0,6076429.0
2022-11-09 00:00:00,부산광역시,8.0,14.0,2879908,2851376.0,2879900,2851362.0,10161,136.0,2140909.0,2140773.0
2022-11-09 00:00:00,대구광역시,8.0,8.0,2018612,1995791.0,2018604,1995783.0,10162,56.0,1405219.0,1405163.0


In [6]:
vac_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10158 entries, 2022-11-09 00:00:00 to 2021-03-11 00:00:00
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sido                  10158 non-null  object 
 1   firstCnt              9839 non-null   float64
 2   secondCnt             9339 non-null   float64
 3   totalFirstCnt         10158 non-null  int64  
 4   totalSecondCnt        9867 non-null   float64
 5   accumulatedFirstCnt   10158 non-null  int64  
 6   accumulatedSecondCnt  9851 non-null   float64
 7   id                    10158 non-null  int64  
 8   thirdCnt              6191 non-null   float64
 9   totalThirdCnt         6261 non-null   float64
 10  accumulatedThirdCnt   6243 non-null   float64
dtypes: float64(7), int64(3), object(1)
memory usage: 952.3+ KB


In [ ]:
- 백신 데이터 광역시도명이 기 수집한 데이터의 광역시도명과 다르기 때문에
    - 백신데이터를 변경 후 파일 저장

In [20]:
# 원본 data추출: 의미없이 반복문 안돌게 중복 없이 추출
vac_v1[['sido']].drop_duplicates()['sido']

baseDate
2022-11-09 00:00:00    제주특별자치도
2022-11-09 00:00:00         전국
2022-11-09 00:00:00      서울특별시
2022-11-09 00:00:00      부산광역시
2022-11-09 00:00:00      대구광역시
2022-11-09 00:00:00      광주광역시
2022-11-09 00:00:00      인천광역시
2022-11-09 00:00:00      대전광역시
2022-11-09 00:00:00        경기도
2022-11-09 00:00:00    세종특별자치시
2022-11-09 00:00:00       경상남도
2022-11-09 00:00:00       경상북도
2022-11-09 00:00:00       전라남도
2022-11-09 00:00:00       전라북도
2022-11-09 00:00:00       충청남도
2022-11-09 00:00:00       충청북도
2022-11-09 00:00:00        강원도
2022-11-09 00:00:00      울산광역시
2021-04-26 00:00:00         기타
Name: sido, dtype: object

In [18]:
before = [sido for sido in vac_v1[['sido']].drop_duplicates()['sido']]
print(before)
before.sort()
before.remove('기타')
print(before)

['제주특별자치도', '전국', '서울특별시', '부산광역시', '대구광역시', '광주광역시', '인천광역시', '대전광역시', '경기도', '세종특별자치시', '경상남도', '경상북도', '전라남도', '전라북도', '충청남도', '충청북도', '강원도', '울산광역시', '기타']
['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전국', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']


In [9]:
pop['loc']

0     전국
1     서울
2     부산
3     대구
4     인천
5     광주
6     대전
7     울산
8     세종
9     경기
10    강원
11    충북
12    충남
13    전북
14    전남
15    경북
16    경남
17    제주
Name: loc, dtype: object

In [19]:
after = [sido for sido in pop['loc']]
after.sort()
print(after)

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전국', '전남', '전북', '제주', '충남', '충북']


In [20]:
for i in range(0, len(after)):
    vac_v1['sido'] = vac_v1['sido'].replace(before[i], after[i])

In [21]:
vac_v1.head(5)

,sido,firstCnt,secondCnt,totalFirstCnt,totalSecondCnt,accumulatedFirstCnt,accumulatedSecondCnt,id,thirdCnt,totalThirdCnt,accumulatedThirdCnt
baseDate,,,,,,,,,,,
2022-11-09 00:00:00,제주,6.0,5.0,591064,585442.0,591058,585437.0,10176,38.0,439144.0,439106.0
2022-11-09 00:00:00,전국,175.0,224.0,45123225,44693321.0,45123050,44693097.0,10159,1678.0,33673146.0,33671468.0
2022-11-09 00:00:00,서울,28.0,43.0,8335326,8257811.0,8335298,8257768.0,10160,482.0,6076911.0,6076429.0
2022-11-09 00:00:00,부산,8.0,14.0,2879908,2851376.0,2879900,2851362.0,10161,136.0,2140909.0,2140773.0
2022-11-09 00:00:00,대구,8.0,8.0,2018612,1995791.0,2018604,1995783.0,10162,56.0,1405219.0,1405163.0


- 2022-01-18에 해당하는 데이터를 vac_v1에서 추출

In [37]:
cal_std_day(365*3)

'2022-01-18'

In [22]:
# T/F를 이용한 논리 indexing
# loc에 논리 인덱싱 행 index만 구성하면 레코드를 추출
finVac = vac_v1.loc[vac_v1.index.str.contains(cal_std_day(365*3))]
finVac.head(2)

,sido,firstCnt,secondCnt,totalFirstCnt,totalSecondCnt,accumulatedFirstCnt,accumulatedSecondCnt,id,thirdCnt,totalThirdCnt,accumulatedThirdCnt
baseDate,,,,,,,,,,,
2022-01-18 00:00:00,광주,523.0,2469.0,1247764,1220758.0,1247241,1218289.0,5648,11353.0,682394.0,671041.0
2022-01-18 00:00:00,울산,382.0,2280.0,956895,933311.0,956513,931031.0,5647,9470.0,476692.0,467222.0


In [ ]:
# duf dntjs dlseprtlr
vax_v1[1]
# 행우선 인덱서: loc[], iloc[]: 인덱서처럼 사용하기 위해서 대괄호 이용하여 사용하는 모듈이다.
# 열우선vac_va['sido',1:3] : 행을 가져오고싶어서 이런코딩하면 출력 불가


#### Json 구성

In [23]:
# 레코드 추출 - iloc[위치값] (행추출)
finVac.iloc[4]

sido                           대구
firstCnt                    893.0
secondCnt                  4524.0
totalFirstCnt             1997407
totalSecondCnt          1945813.0
accumulatedFirstCnt       1996514
accumulatedSecondCnt    1941289.0
id                           5644
thirdCnt                  15716.0
totalThirdCnt            978794.0
accumulatedThirdCnt      963078.0
Name: 2022-01-18 00:00:00, dtype: object

In [24]:
# hdfs에 저장할 data
cols = ['loc', 'v1', 'v2', 'v3']
data = []

In [105]:
# for idx in range (0, len(finVac.index)):
#     tuple_t=[]
#     tmp = finVac.iloc[idx]
#     tuple_t.append(str(tmp.sido))
#     tuple_t.append(str(tmp.totalFirstCnt))
#     tuple_t.append(str(tmp.totalSecondCnt))
#     tuple_t.append(str(tmp.totalThirdCnt))
#     print(tuple_t)

['광주', '1247764', '1220758.0', '682394.0']
['울산', '956895', '933311.0', '476692.0']
['인천', '2532911', '2484196.0', '1330273.0']
['대전', '1231435', '1203724.0', '628469.0']
['대구', '1997407', '1945813.0', '978794.0']
['부산', '2849093', '2788089.0', '1554554.0']
['서울', '8239400', '8090244.0', '4304485.0']
['전국', '44505276', '43582128.0', '23741205.0']
['세종', '290455', '282507.0', '138100.0']
['경기', '11652853', '11414202.0', '5970505.0']
['제주', '580579', '566811.0', '295807.0']
['경남', '2848083', '2783896.0', '1517240.0']
['경북', '2260963', '2202814.0', '1247047.0']
['전남', '1632045', '1602479.0', '1031459.0']
['전북', '1574779', '1547262.0', '958100.0']
['충남', '1870124', '1830143.0', '1048429.0']
['충북', '1408620', '1379378.0', '783588.0']
['강원', '1331870', '1306501.0', '795269.0']


In [25]:
# 얘를 위에 hdfs에 저장할 data로 만들어 놓은 cols 와 인덱스 별로 매칭시켜서 데이터 구성 ==> zip함수 이용
for idx in range (0, len(finVac.index)):
    tuple_t=[]
    tmp = finVac.iloc[idx]
    tuple_t.append(str(tmp.sido))
    tuple_t.append(str(tmp.totalFirstCnt))
    tuple_t.append(str(tmp.totalSecondCnt))
    tuple_t.append(str(tmp.totalThirdCnt))
    data.append(dict(zip(cols, tuple_t)))

In [26]:
# 실제 data
data

[{'loc': '광주', 'v1': '1247764', 'v2': '1220758.0', 'v3': '682394.0'},
 {'loc': '울산', 'v1': '956895', 'v2': '933311.0', 'v3': '476692.0'},
 {'loc': '인천', 'v1': '2532911', 'v2': '2484196.0', 'v3': '1330273.0'},
 {'loc': '대전', 'v1': '1231435', 'v2': '1203724.0', 'v3': '628469.0'},
 {'loc': '대구', 'v1': '1997407', 'v2': '1945813.0', 'v3': '978794.0'},
 {'loc': '부산', 'v1': '2849093', 'v2': '2788089.0', 'v3': '1554554.0'},
 {'loc': '서울', 'v1': '8239400', 'v2': '8090244.0', 'v3': '4304485.0'},
 {'loc': '전국', 'v1': '44505276', 'v2': '43582128.0', 'v3': '23741205.0'},
 {'loc': '세종', 'v1': '290455', 'v2': '282507.0', 'v3': '138100.0'},
 {'loc': '경기', 'v1': '11652853', 'v2': '11414202.0', 'v3': '5970505.0'},
 {'loc': '제주', 'v1': '580579', 'v2': '566811.0', 'v3': '295807.0'},
 {'loc': '경남', 'v1': '2848083', 'v2': '2783896.0', 'v3': '1517240.0'},
 {'loc': '경북', 'v1': '2260963', 'v2': '2202814.0', 'v3': '1247047.0'},
 {'loc': '전남', 'v1': '1632045', 'v2': '1602479.0', 'v3': '1031459.0'},
 {'loc': '전북'

In [16]:
# data를 설명하는 메타 데이터 포함한 최종 결과 생성
res = {
    'meta' : {
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
            'loc':'지역'
            ,'v1':'1차 접종 누적'
            ,'v2':'2차 접종 누적'
            ,'v3':'3차 접종 누적'
        },
        'stdDay':cal_std_day(365*3)
    },
    'data':data
}

NameError: name 'data' is not defined

In [28]:
res

{'meta': {'desc': '지역별 코로나 예방접종 인구 현황',
  'cols': {'loc': '지역', 'v1': '1차 접종 누적', 'v2': '2차 접종 누적', 'v3': '3차 접종 누적'},
  'stdDay': '2022-01-18'},
 'data': [{'loc': '광주', 'v1': '1247764', 'v2': '1220758.0', 'v3': '682394.0'},
  {'loc': '울산', 'v1': '956895', 'v2': '933311.0', 'v3': '476692.0'},
  {'loc': '인천', 'v1': '2532911', 'v2': '2484196.0', 'v3': '1330273.0'},
  {'loc': '대전', 'v1': '1231435', 'v2': '1203724.0', 'v3': '628469.0'},
  {'loc': '대구', 'v1': '1997407', 'v2': '1945813.0', 'v3': '978794.0'},
  {'loc': '부산', 'v1': '2849093', 'v2': '2788089.0', 'v3': '1554554.0'},
  {'loc': '서울', 'v1': '8239400', 'v2': '8090244.0', 'v3': '4304485.0'},
  {'loc': '전국', 'v1': '44505276', 'v2': '43582128.0', 'v3': '23741205.0'},
  {'loc': '세종', 'v1': '290455', 'v2': '282507.0', 'v3': '138100.0'},
  {'loc': '경기', 'v1': '11652853', 'v2': '11414202.0', 'v3': '5970505.0'},
  {'loc': '제주', 'v1': '580579', 'v2': '566811.0', 'v3': '295807.0'},
  {'loc': '경남', 'v1': '2848083', 'v2': '2783896.0', 'v3': '15

In [29]:
file_dir_vac = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365*3)+'.json'
client.write(file_dir_vac+file_name,json.dumps(res, ensure_ascii=False),encoding='utf-8')

In [30]:
# spark 모듈로 방금 저장한 파일 읽어오기
# read.json: df로 지연연산
file_dir_vac = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365*3)+'.json'
vaccine=spark.read.json(file_dir_vac+file_name)
vaccine.show()

[Stage 1:>                                                          (0 + 1) / 1]

+----------------------+---------------------------+
|                  data|                       meta|
+----------------------+---------------------------+
|[{광주, 1247764, 12...|{{지역, 1차 접종 누적, 2...|
+----------------------+---------------------------+



In [32]:
### 파일 읽어오기
vac_v1 = pd.read_csv('data/corona_data/Covid19VaccineV2.csv', index_col=0)

In [33]:
# 전체 코드
cols = ['loc', 'v1', 'v2', 'v3']
data = []
finVac = vac_v1.loc[vac_v1.index.str.contains(cal_std_day(365*3))]

for idx in range (0, len(finVac.index)):
    tuple_t=[]
    tmp = finVac.iloc[idx]
    tuple_t.append(str(tmp.sido))
    tuple_t.append(str(tmp.totalFirstCnt))
    tuple_t.append(str(tmp.totalSecondCnt))
    tuple_t.append(str(tmp.totalThirdCnt))
    data.append(dict(zip(cols, tuple_t)))

res = {
    'meta' : {
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
            'loc':'지역'
            ,'v1':'1차 접종 누적'
            ,'v2':'2차 접종 누적'
            ,'v3':'3차 접종 누적'
        },
        'stdDay':cal_std_day(365*3)
    },
    'data':data
}

file_dir_vac = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365*3)+'.json'
client.write(file_dir_vac+file_name,json.dumps(res, ensure_ascii=False),encoding='utf-8')

HdfsError: /corona_data/vaccine/corona_vaccine_2022-01-18.json for client 127.0.0.1 already exists
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.startFile(FSDirWriteFileOp.java:388)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFileInt(FSNamesystem.java:2819)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFile(FSNamesystem.java:2713)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.create(NameNodeRpcServer.java:830)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.create(ClientNamenodeProtocolServerSideTranslatorPB.java:504)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngine2.java:621)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngine2.java:589)
	at org.apache.hadoo

In [34]:
# 연습용 파일 생성 -> hdfs에 저장

for i in range(365*3, 367*3):
    print(cal_std_day(i))
    cols = ['loc', 'v1', 'v2', 'v3']
    data = []
    finVac = vac_v1.loc[vac_v1.index.str.contains(cal_std_day(i))]
    
    for idx in range (0, len(finVac.index)):
        tuple_t=[]
        tmp = finVac.iloc[idx]
        tuple_t.append(str(tmp.sido))
        tuple_t.append(str(tmp.totalFirstCnt))
        tuple_t.append(str(tmp.totalSecondCnt))
        tuple_t.append(str(tmp.totalThirdCnt))
        data.append(dict(zip(cols, tuple_t)))
    
    res = {
        'meta' : {
            'desc':'지역별 코로나 예방접종 인구 현황',
            'cols':{
                'loc':'지역'
                ,'v1':'1차 접종 누적'
                ,'v2':'2차 접종 누적'
                ,'v3':'3차 접종 누적'
            },
            'stdDay':cal_std_day(i)
        },
        'data':data
    }
    try:
        file_dir_vac = '/corona_data/vaccine/'
        file_name = 'corona_vaccine_'+cal_std_day(i)+'.json'
        client.write(file_dir_vac+file_name,json.dumps(res, ensure_ascii=False),encoding='utf-8')
    except Exception as e:
        log_dict = {
            'type':'/corona_data/vaccine/'
            ,'error_message' : e.__str__()}
        log_json = json.dumps(log_dict, ensure_ascii=False)
        co_logger.error(log_json)

2022-01-18
2022-01-17
2022-01-16
2022-01-15
2022-01-14
2022-01-13
